In [4]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import glob

with open("elt_cypher_queries/cleanup_nodes.cypher") as f:
    cleanup_node_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_edges.cypher") as f:
    cleanup_edge_queries = [line for line in f]

with open("elt_cypher_queries/index_nodes.cypher") as f:
    node_index_queries = [line for line in f]

node_cypher_files = glob.glob("elt_cypher_queries/nodes/*.cypher")
node_creation_queries = []
for query_file in node_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        node_creation_queries.append(creation_query)

edge_cypher_files = glob.glob("elt_cypher_queries/edges/*.cypher")
edge_creation_queries = []
for query_file in edge_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        edge_creation_queries.append(creation_query)

In [19]:
from neo4j import GraphDatabase

class IMDBELTLoader:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, params_dict={}):
        with self.driver.session(database="neo4j") as session:
            result = session.run(query, **params_dict)
     
# for query in cleanup_edge_queries+cleanup_node_queries:
#     print(f"Processing {query}")
#     loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
#     loader.execute_query(query)
#     loader.close()

# for query in node_creation_queries+node_index_queries+edge_creation_queries:
#     print(f"Processing {query}")
#     loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
#     loader.execute_query(query)
#     loader.close()

for query in cleanup_edge_queries:
    print(f"Processing {query}")
    loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
    loader.execute_query(query)
    loader.close()

for query in node_index_queries+edge_creation_queries:
    print(f"Processing {query}")
    loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
    loader.execute_query(query)
    loader.close()


Processing MATCH ()-[r]->() CALL { WITH r DETACH DELETE r} IN TRANSACTIONS OF 10000 ROWS;
Processing CREATE CONSTRAINT imdb_attribute_attribute IF NOT EXISTS FOR (n:IMDB_ATTRIBUTE) REQUIRE n.attribute IS UNIQUE;

Processing CREATE CONSTRAINT imdb_characters_somecharacter IF NOT EXISTS FOR (n:IMDB_CHARACTERS) REQUIRE n.attribute IS UNIQUE;

Processing CREATE CONSTRAINT imdb_distribution_type_distribution_type IF NOT EXISTS FOR (n:IMDB_DISTRIBUTION_TYPE) REQUIRE n.attribute IS UNIQUE;

Processing CREATE CONSTRAINT imdb_genre_genre IF NOT EXISTS FOR (n:IMDB_GENRE) REQUIRE n.genre IS UNIQUE;

Processing CREATE CONSTRAINT imdb_job_category_category IF NOT EXISTS FOR (n:IMDB_JOB_CATEGORY) REQUIRE n.category IS UNIQUE;

Processing CREATE CONSTRAINT imdb_language_code_alpha2 IF NOT EXISTS FOR (n:IMDB_LANGUAGE_CODE) REQUIRE n.alpha2 IS UNIQUE;

Processing CREATE CONSTRAINT imdb_name_basics_nconst IF NOT EXISTS FOR (n:IMDB_NAME_BASICS) REQUIRE n.nconst IS UNIQUE;

Processing CREATE CONSTRAINT im